In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

In [2]:
df = pd.read_excel('Sentiment_DB.xlsx', engine='openpyxl')

In [3]:
df

,Reported_Date,Bullish,Neutral,Bearish,Bearish_8-week_MA,Bullish_8-week_MA,Bull-Bear_Spread,S&P500_Intraweek_spread,S&P500_Weekly_High,S&P500_Weekly_Low,S&P500_Weekly_return,S&P500_1_Week_forward,S&P500_4_Week_forward,S&P500_8_Week_forward,S&P500_Weekly_High.1,S&P500_Weekly_Low.1,S&P500_Weekly_Close
0,1987-06-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1987-07-17,NaN,NaN,NaN,NaN,NaN,NaN,0.022124,0.000000,-0.022124,NaN,-0.016911,0.026733,0.026860,314.59,307.63,314.59
2,1987-07-24,0.360000,0.500000,0.140000,NaN,NaN,0.220000,0.011576,0.006855,-0.004721,-0.016911,0.030362,0.079930,0.024024,311.39,307.81,309.27
3,1987-07-31,0.260000,0.480000,0.260000,NaN,NaN,0.000000,0.025137,0.000000,-0.025137,0.030362,0.013620,0.054102,0.010419,318.66,310.65,318.66
4,1987-08-07,0.560000,0.150000,0.290000,NaN,NaN,0.270000,0.020960,0.000000,-0.020960,0.013620,0.034025,0.000124,-0.025201,323.00,316.23,323.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804,2022-02-24,0.233945,0.229358,0.536697,0.434098,0.244073,-0.302752,0.066253,0.040019,-0.026234,-0.055756,0.038111,0.031324,NaN,4394.60,4114.65,4225.50
1805,2022-03-03,0.303867,0.281768,0.414365,0.444227,0.241108,-0.110498,0.031287,0.006894,-0.024393,0.038111,-0.024771,0.015890,NaN,4416.78,4279.54,4386.54
1806,2022-03-10,0.240000,0.302222,0.457778,0.453587,0.239974,-0.217778,0.043070,0.015017,-0.028054,-0.024771,0.018696,NaN,NaN,4342.12,4157.87,4277.88
1807,2022-03-17,0.224670,0.277533,0.497797,0.457405,0.241857,-0.273127,0.048246,0.003238,-0.045008,0.018696,0.022575,NaN,NaN,4371.97,4161.72,4357.86


In [4]:
ma_period = 4
df[f'Bearish_{ma_period}-week_MA'] = df['Bearish'].rolling(ma_period, min_periods=ma_period, center=False).mean()
df[f'Bullish_{ma_period}-week_MA'] = df['Bullish'].rolling(ma_period, min_periods=ma_period, center=False).mean()
df[f'Bull-Bear_Spread_{ma_period}-week_MA'] = df['Bull-Bear_Spread'].rolling(ma_period, min_periods=ma_period, center=False).mean()
df[f'S&P500_Intraweek_spread_{ma_period}-week_MA'] = df['S&P500_Intraweek_spread'].rolling(ma_period, min_periods=ma_period, center=False).mean()
df[f'S&P500_Weekly_High_{ma_period}-week_MA'] = df['S&P500_Weekly_High'].rolling(ma_period, min_periods=ma_period, center=False).mean()
df[f'S&P500_Weekly_Low_{ma_period}-week_MA'] = df['S&P500_Weekly_Low'].rolling(ma_period, min_periods=ma_period, center=False).mean()
# df['Percentage_with_opinion'] = 1-df['Neutral']

In [5]:
df

,Reported_Date,Bullish,Neutral,Bearish,Bearish_8-week_MA,Bullish_8-week_MA,Bull-Bear_Spread,S&P500_Intraweek_spread,S&P500_Weekly_High,S&P500_Weekly_Low,...,S&P500_8_Week_forward,S&P500_Weekly_High.1,S&P500_Weekly_Low.1,S&P500_Weekly_Close,Bearish_4-week_MA,Bullish_4-week_MA,Bull-Bear_Spread_4-week_MA,S&P500_Intraweek_spread_4-week_MA,S&P500_Weekly_High_4-week_MA,S&P500_Weekly_Low_4-week_MA
0,1987-06-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1987-07-17,NaN,NaN,NaN,NaN,NaN,NaN,0.022124,0.000000,-0.022124,...,0.026860,314.59,307.63,314.59,NaN,NaN,NaN,NaN,NaN,NaN
2,1987-07-24,0.360000,0.500000,0.140000,NaN,NaN,0.220000,0.011576,0.006855,-0.004721,...,0.024024,311.39,307.81,309.27,NaN,NaN,NaN,NaN,NaN,NaN
3,1987-07-31,0.260000,0.480000,0.260000,NaN,NaN,0.000000,0.025137,0.000000,-0.025137,...,0.010419,318.66,310.65,318.66,NaN,NaN,NaN,NaN,NaN,NaN
4,1987-08-07,0.560000,0.150000,0.290000,NaN,NaN,0.270000,0.020960,0.000000,-0.020960,...,-0.025201,323.00,316.23,323.00,NaN,NaN,NaN,0.019949,0.001714,-0.018235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804,2022-02-24,0.233945,0.229358,0.536697,0.434098,0.244073,-0.302752,0.066253,0.040019,-0.026234,...,NaN,4394.60,4114.65,4225.50,0.439972,0.233737,-0.206236,0.047868,0.013350,-0.034518
1805,2022-03-03,0.303867,0.281768,0.414365,0.444227,0.241108,-0.110498,0.031287,0.006894,-0.024393,...,NaN,4416.78,4279.54,4386.54,0.434422,0.243473,-0.190950,0.039192,0.014751,-0.024442
1806,2022-03-10,0.240000,0.302222,0.457778,0.453587,0.239974,-0.217778,0.043070,0.015017,-0.028054,...,NaN,4342.12,4157.87,4277.88,0.460191,0.242575,-0.217616,0.044174,0.018349,-0.025825
1807,2022-03-17,0.224670,0.277533,0.497797,0.457405,0.241857,-0.273127,0.048246,0.003238,-0.045008,...,NaN,4371.97,4161.72,4357.86,0.476659,0.250620,-0.226039,0.047214,0.016292,-0.030922


In [6]:
df.dropna(inplace=True)
df

,Reported_Date,Bullish,Neutral,Bearish,Bearish_8-week_MA,Bullish_8-week_MA,Bull-Bear_Spread,S&P500_Intraweek_spread,S&P500_Weekly_High,S&P500_Weekly_Low,...,S&P500_8_Week_forward,S&P500_Weekly_High.1,S&P500_Weekly_Low.1,S&P500_Weekly_Close,Bearish_4-week_MA,Bullish_4-week_MA,Bull-Bear_Spread_4-week_MA,S&P500_Intraweek_spread_4-week_MA,S&P500_Weekly_High_4-week_MA,S&P500_Weekly_Low_4-week_MA
9,1987-09-11,0.500000,0.230000,0.270000,0.241250,0.466250,0.230000,0.026151,0.000000,-0.026151,...,-0.229083,321.98,313.56,321.98,0.260000,0.525000,0.265000,0.028274,0.016787,-0.011487
10,1987-09-18,0.600000,0.290000,0.110000,0.237500,0.496250,0.490000,0.026107,0.026107,0.000000,...,-0.200311,323.08,314.86,314.86,0.272500,0.510000,0.237500,0.029852,0.023314,-0.006538
11,1987-09-25,0.530000,0.170000,0.300000,0.242500,0.530000,0.230000,0.033265,0.003217,-0.030047,...,-0.217860,321.19,310.54,320.16,0.272500,0.512500,0.240000,0.031714,0.017664,-0.014050
12,1987-10-02,0.440000,0.260000,0.300000,0.243750,0.515000,0.140000,0.019447,0.000000,-0.019447,...,-0.251257,328.07,321.69,328.07,0.245000,0.517500,0.272500,0.026242,0.007331,-0.018911
13,1987-10-09,0.460000,0.390000,0.150000,0.237500,0.516250,0.310000,0.054682,0.054682,0.000000,...,-0.222040,328.08,311.07,311.07,0.215000,0.507500,0.292500,0.033375,0.021002,-0.012374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1798,2022-01-13,0.249071,0.368030,0.382900,0.354970,0.300413,-0.133829,0.023392,0.004756,-0.018636,...,-0.105970,4748.83,4638.27,4726.35,0.340095,0.312231,-0.027864,0.027714,0.009540,-0.018174
1799,2022-01-20,0.209607,0.323144,0.467249,0.368733,0.284352,-0.257642,0.012081,0.015300,0.003219,...,-0.032258,4602.11,4547.35,4532.76,0.372230,0.290708,-0.081522,0.020582,0.012020,-0.008562
1800,2022-01-27,0.231373,0.239216,0.529412,0.381933,0.279940,-0.298039,0.053015,0.023748,-0.029267,...,-0.016563,4453.23,4222.62,4349.93,0.428223,0.254409,-0.173814,0.028461,0.017229,-0.011232
1801,2022-02-03,0.264925,0.298507,0.436567,0.398352,0.275907,-0.171642,0.065989,0.001292,-0.064697,...,-0.050447,4595.31,4292.46,4589.38,0.454032,0.238744,-0.215288,0.038619,0.011274,-0.027345


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1794 entries, 9 to 1802
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Reported_Date                      1794 non-null   datetime64[ns]
 1   Bullish                            1794 non-null   float64       
 2   Neutral                            1794 non-null   float64       
 3   Bearish                            1794 non-null   float64       
 4   Bearish_8-week_MA                  1794 non-null   float64       
 5   Bullish_8-week_MA                  1794 non-null   float64       
 6   Bull-Bear_Spread                   1794 non-null   float64       
 7   S&P500_Intraweek_spread            1794 non-null   float64       
 8   S&P500_Weekly_High                 1794 non-null   float64       
 9   S&P500_Weekly_Low                  1794 non-null   float64       
 10  S&P500_Weekly_return               1

## Train-test split

In [8]:
features = ['Bullish', 'Bearish', 'Bull-Bear_Spread', 'S&P500_Intraweek_spread', 'S&P500_Weekly_High', 'S&P500_Weekly_Low', 'S&P500_Weekly_return']
for col in df.columns:
    if 'MA' in col:
        features.append(col)
len(features)

15

In [9]:
# features = ['Bullish', 'Bearish']

In [10]:
X = np.array(df[features])
X.shape

(1794, 15)

In [11]:
y = np.array(df['S&P500_4_Week_forward'])

In [12]:
days_for_test = 1
# Split the data into training/testing sets
X_train = X[:-days_for_test]
X_test = X[-days_for_test:]

In [13]:
# Split the targets into training/testing sets
y_train = y[:-days_for_test]
y_test = y[-days_for_test:]

## Linear regression

In [14]:
# Create linear regression object
regr = linear_model.LinearRegression()

In [15]:
# Train the model using the training sets
regr.fit(X_train, y_train)

LinearRegression()

In [16]:
# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [17]:
# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("MSE: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("R^2: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [ 0.01424097  0.0083846   0.00585638 -0.00393918 -0.02476577 -0.02082658
 -0.03184684  0.0842019   0.04669625 -0.09157625 -0.09819759 -0.00662135
  0.15597284  0.31448519  0.15851235]
MSE: 0.00
R^2: nan


C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:682: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [18]:
regression_result = pd.DataFrame([features, list(regr.coef_)]).T
regression_result.columns = ['Feature','Coefficient']
regression_result.set_index('Feature')

,Coefficient
Feature,
Bullish,0.014241
Bearish,0.008385
Bull-Bear_Spread,0.005856
S&P500_Intraweek_spread,-0.003939
S&P500_Weekly_High,-0.024766
S&P500_Weekly_Low,-0.020827
S&P500_Weekly_return,-0.031847
Bearish_8-week_MA,0.084202
Bullish_8-week_MA,0.046696


## Linear regression with p-value

In [19]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     4.035
Date:                Mon, 11 Apr 2022   Prob (F-statistic):           7.41e-06
Time:                        18:24:22   Log-Likelihood:                 3370.5
No. Observations:                1793   AIC:                            -6717.
Df Residuals:                    1781   BIC:                            -6651.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0140      0.009      1.574      0.116      -0.003       0.031
x1             0.0142      0.021      0.669      0.504      -0.028       0.056
x2             0.0084      0.022      0.382      0.703      -0.035       0.051
x3             0.0059      0.006      0.954      0.340      -0.006       0.018
x4            -0.0039      0.059     -0.067      0.946      -0.119       0.111
x5            -0.0248      0.073     -0.341      0.733      -0.167       0.118
x6            -0.0208      0.067     -0.311      0.756      -0.152       0.110
x7            -0.0318      0.059     -0.543      0.587      -0.147       0.083
x8             0.0842      0.046      1.829      0.068      -0.006       0.175
x9             0.0467      0.042      1.115      0.265      -0.035       0.129
x10           -0.0916      0.047     -1.948      0.052      -0.184       0.001
x11           -0.0982      0.046     -2.156      0.031      -0.188      -0.009
x12           -0.0066      0.012     -0.554      0.579      -0.030       0.017
x13            0.1560      0.068      2.302      0.021       0.023       0.289
x14            0.3145      0.113      2.784      0.005       0.093       0.536
x15            0.1585      0.104      1.519      0.129      -0.046       0.363
==============================================================================
Omnibus:                      572.547   Durbin-Watson:                   0.702
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3554.936
Skew:                          -1.351   Prob(JB):                         0.00
Kurtosis:                       9.347   Cond. No.                     7.95e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.9e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [20]:
# The R^2 of train

In [21]:
y_trained = regr.predict(X_train)
y_trained
print("R^2: %.2f" % r2_score(y_train, y_trained))

R^2: 0.02


that means the model is not flexible enough

## Desision tree